In [3]:
import os
import subprocess
import numpy as np
from Bio import PDB
from Bio.PDB.Superimposer import Superimposer


In [15]:
# Cell 1: Import necessary libraries
import os
import subprocess
import numpy as np
from Bio import PDB
from Bio.PDB.PDBList import PDBList
from Bio.PDB.Superimposer import Superimposer
import warnings
from Bio.PDB.PDBExceptions import PDBConstructionWarning



In [18]:
# Cell 2: Custom warning handler
def custom_warning_handler(message, category, filename, lineno, file=None, line=None):
    print(f"Warning: {message}")

warnings.showwarning = custom_warning_handler


In [19]:
# Cell 2: Process PDB files with mark_sur
#def run_mark_sur(input_pdb, output_pdb):
#    subprocess.run([mark_sur_path, input_pdb, output_pdb], check=True)

# Cell 2: Download PDB files
def download_pdb(pdb_id):
    pdbl = PDBList()
    filename = pdbl.retrieve_pdb_file(pdb_id, pdir='.', file_format='pdb')
    return filename
# Download the crystal structure
crystal_file = download_pdb('1ACB')


Structure exists: './pdb1acb.ent' 


In [20]:
# Cell 3: Run ZDOCK
#def run_zdock(receptor, ligand, output):
#    subprocess.run([zdock_path, "-R", receptor, "-L", ligand, "-o", output], check=True)
# Cell 3: Load structures
def load_structure(filename):
    parser = PDB.PDBParser()
    structure = parser.get_structure('structure', filename)
    return structure

crystal_structure = load_structure(crystal_file)


In [21]:
# Cell 4: Create complex structures
#def create_complexes(zdock_out):
#    subprocess.run([create_pl_path, zdock_out], check=True)
# Cell 4: Load structures with additional information
def load_structure(filename):
    parser = PDB.PDBParser(QUIET=True)
    structure = parser.get_structure('structure', filename)
    
    # Analyze chain continuity
    for model in structure:
        for chain in model:
            residues = list(chain.get_residues())
            for i in range(len(residues) - 1):
                if residues[i].id[1] + 1 != residues[i+1].id[1]:
                    print(f"Gap detected in chain {chain.id} between residues {residues[i].id[1]} and {residues[i+1].id[1]}")
    
    return structure


In [22]:
# Cell 4a : Load PDB structure
#def load_structure(filename):
#    parser = PDB.PDBParser()
#    structure = parser.get_structure('structure', filename)
#    return structure
# Load the crystal structure
with warnings.catch_warnings(record=True) as w:
    warnings.simplefilter("always")
    crystal_structure = load_structure(crystal_file)
    for warning in w:
        print(f"Warning details: {warning}")


Gap detected in chain E between residues 13 and 16
Gap detected in chain E between residues 146 and 149
Gap detected in chain E between residues 245 and 406
Gap detected in chain E between residues 421 and 423
Gap detected in chain E between residues 425 and 427
Gap detected in chain E between residues 446 and 448
Gap detected in chain E between residues 459 and 463
Gap detected in chain E between residues 463 and 469
Gap detected in chain E between residues 469 and 501
Gap detected in chain E between residues 504 and 516
Gap detected in chain E between residues 518 and 523
Gap detected in chain E between residues 525 and 527
Gap detected in chain E between residues 528 and 532
Gap detected in chain E between residues 532 and 535
Gap detected in chain E between residues 535 and 538
Gap detected in chain E between residues 538 and 601
Gap detected in chain E between residues 601 and 603
Gap detected in chain E between residues 604 and 606
Gap detected in chain E between residues 608 and

In [23]:
# make sure you have pymol installed in order to visualize the protein and the gaps detected in the previous cell
!pymol pdb1acb.ent

 PyMOL(TM) Molecular Graphics System, Version 3.0.0.
 Copyright (c) Schrodinger, LLC.
 All Rights Reserved.
 
    Created by Warren L. DeLano, Ph.D. 
 
    PyMOL is user-supported open-source software.  Although some versions
    are freely available, PyMOL is not in the public domain.
 
    If PyMOL is helpful in your work or study, then please volunteer 
    support for our ongoing efforts to create open and affordable scientific
    software by purchasing a PyMOL Maintenance and/or Support subscription.

    More information can be found at "http://www.pymol.org".
 
    Enter "help" for a list of commands.
    Enter "help <command-name>" for information on a specific command.

 Hit ESC anytime to toggle between text and graphics.

 Detected OpenGL version 2.1. Shaders available.
 Tessellation shaders not available
 Detected GLSL version 1.20.
 OpenGL graphics engine:
  GL_VENDOR:   Apple
  GL_RENDERER: Apple M2 Pro
  GL_VERSION:  2.1 Metal - 88.1
 Detected 10 CPU cores.  Enabled mul

In [24]:
# Cell 6: Compare models
#def compare_models(models):
#    rmsd_matrix = np.zeros((len(models), len(models)))
#    for i in range(len(models)):
#        for j in range(i+1, len(models)):
#            sup = Superimposer()
#            atoms_1 = list(models[i].get_atoms())
#            atoms_2 = list(models[j].get_atoms())
#            sup.set_atoms(atoms_1, atoms_2)
#            rmsd_matrix[i,j] = rmsd_matrix[j,i] = sup.rms
#    return rmsd_matrix

# Cell 5: Analyze structure
def analyze_structure(structure):
    print(f"Structure ID: {structure.id}")
    print(f"Number of models: {len(structure)}")
    for model in structure:
        print(f"  Model {model.id}:")
        for chain in model:
            residues = list(chain.get_residues())
            print(f"    Chain {chain.id}: {len(residues)} residues")
            print(f"      First residue: {residues[0].resname} {residues[0].id[1]}")
            print(f"      Last residue: {residues[-1].resname} {residues[-1].id[1]}")

print("Analysis of crystal structure:")
analyze_structure(crystal_structure)



Analysis of crystal structure:
Structure ID: structure
Number of models: 1
  Model 0:
    Chain E: 354 residues
      First residue: CYS 1
      Last residue: HOH 726
    Chain I: 92 residues
      First residue: LYS 8
      Last residue: HOH 725


# In the following section of the notebook we will create a script that provides a comprehensive workflow for comparing protein structures, preparing them for docking, running ZDOCK, and analyzing the results. 

In [25]:
# Cell 1: Import necessary libraries
import os
from Bio import PDB
from Bio.PDB.PDBList import PDBList
from Bio.PDB.Superimposer import Superimposer
import numpy as np


In [26]:
# Cell 2: Download and load PDB structures
def download_and_load_pdb(pdb_id):
    pdbl = PDBList()
    filename = pdbl.retrieve_pdb_file(pdb_id, pdir='.', file_format='pdb')
    parser = PDB.PDBParser(QUIET=True)
    structure = parser.get_structure(pdb_id, filename)
    return structure

complex_structure = download_and_load_pdb('1ACB')
receptor_structure = download_and_load_pdb('5CHA')
ligand_structure = download_and_load_pdb('1CSE')


Structure exists: './pdb1acb.ent' 


In [29]:
# Cell 3: Function to calculate RMSD between two structures
def calculate_rmsd(struct1, struct2, chain1, chain2):
    atoms1 = [atom for atom in struct1[0][chain1].get_atoms() if atom.name == 'CA']
    atoms2 = [atom for atom in struct2[0][chain2].get_atoms() if atom.name == 'CA']
    
    len1, len2 = len(atoms1), len(atoms2)
    if len1 != len2:
        return None, f"Atom count mismatch: {len1} vs {len2}"
    
    sup = Superimposer()
    sup.set_atoms(atoms1, atoms2)
    return sup.rms, None


In [30]:
# Cell 4: Compare complex structure with individual structures
def compare_structures(struct1, struct2, chain1, chain2, name1, name2):
    rmsd, error = calculate_rmsd(struct1, struct2, chain1, chain2)
    if rmsd is not None:
        print(f"RMSD between {name1} (chain {chain1}) and {name2} (chain {chain2}): {rmsd:.2f} Å")
    else:
        print(f"Could not calculate RMSD between {name1} (chain {chain1}) and {name2} (chain {chain2}): {error}")
    
    # Print additional information about the chains
    print(f"  {name1} (chain {chain1}) atom count: {len(list(struct1[0][chain1].get_atoms()))}")
    print(f"  {name2} (chain {chain2}) atom count: {len(list(struct2[0][chain2].get_atoms()))}")
    print(f"  {name1} (chain {chain1}) residue count: {len(list(struct1[0][chain1].get_residues()))}")
    print(f"  {name2} (chain {chain2}) residue count: {len(list(struct2[0][chain2].get_residues()))}")
    print()

compare_structures(complex_structure, receptor_structure, 'E', 'A', '1ACB', '5CHA')
compare_structures(complex_structure, ligand_structure, 'I', 'I', '1ACB', '1CSE')

Could not calculate RMSD between 1ACB (chain E) and 5CHA (chain A): Atom count mismatch: 241 vs 8
  1ACB (chain E) atom count: 1880
  5CHA (chain A) atom count: 63
  1ACB (chain E) residue count: 354
  5CHA (chain A) residue count: 18

RMSD between 1ACB (chain I) and 1CSE (chain I): 0.63 Å
  1ACB (chain I) atom count: 551
  1CSE (chain I) atom count: 642
  1ACB (chain I) residue count: 92
  1CSE (chain I) residue count: 183



In [31]:
# Cell 5: Detailed structure analysis
def analyze_structure(structure, structure_id):
    print(f"Analyzing {structure_id}:")
    for model in structure:
        for chain in model:
            print(f"  Chain {chain.id}:")
            residues = list(chain.get_residues())
            print(f"    Number of residues: {len(residues)}")
            print(f"    First residue: {residues[0].resname} {residues[0].id[1]}")
            print(f"    Last residue: {residues[-1].resname} {residues[-1].id[1]}")
            print(f"    Number of atoms: {len(list(chain.get_atoms()))}")
    print()

analyze_structure(complex_structure, "1ACB (Complex)")
analyze_structure(receptor_structure, "5CHA (Receptor)")
analyze_structure(ligand_structure, "1CSE (Ligand)")

Analyzing 1ACB (Complex):
  Chain E:
    Number of residues: 354
    First residue: CYS 1
    Last residue: HOH 726
    Number of atoms: 1880
  Chain I:
    Number of residues: 92
    First residue: LYS 8
    Last residue: HOH 725
    Number of atoms: 551

Analyzing 5CHA (Receptor):
  Chain A:
    Number of residues: 18
    First residue: CYS 1
    Last residue: HOH 741
    Number of atoms: 63
  Chain B:
    Number of residues: 206
    First residue: ILE 16
    Last residue: HOH 742
    Number of atoms: 1055
  Chain C:
    Number of residues: 154
    First residue: ALA 149
    Last residue: HOH 739
    Number of atoms: 759
  Chain E:
    Number of residues: 12
    First residue: CYS 1
    Last residue: HOH 693
    Number of atoms: 57
  Chain F:
    Number of residues: 190
    First residue: ILE 16
    Last residue: HOH 738
    Number of atoms: 1039
  Chain G:
    Number of residues: 141
    First residue: ALA 149
    Last residue: HOH 735
    Number of atoms: 746

Analyzing 1CSE (Ligan

In [32]:
# Cell 6: Re-download and analyze 5CHA
def redownload_and_analyze(pdb_id):
    pdbl = PDBList()
    filename = pdbl.retrieve_pdb_file(pdb_id, pdir='.', file_format='pdb', overwrite=True)
    structure = PDB.PDBParser(QUIET=True).get_structure(pdb_id, filename)
    analyze_structure(structure, f"{pdb_id} (Re-downloaded)")

redownload_and_analyze("")

Analyzing 5CHA (Re-downloaded):
  Chain A:
    Number of residues: 18
    First residue: CYS 1
    Last residue: HOH 741
    Number of atoms: 63
  Chain B:
    Number of residues: 206
    First residue: ILE 16
    Last residue: HOH 742
    Number of atoms: 1055
  Chain C:
    Number of residues: 154
    First residue: ALA 149
    Last residue: HOH 739
    Number of atoms: 759
  Chain E:
    Number of residues: 12
    First residue: CYS 1
    Last residue: HOH 693
    Number of atoms: 57
  Chain F:
    Number of residues: 190
    First residue: ILE 16
    Last residue: HOH 738
    Number of atoms: 1039
  Chain G:
    Number of residues: 141
    First residue: ALA 149
    Last residue: HOH 735
    Number of atoms: 746



In [33]:
# Cell 7: Updated structure preparation for docking
def prepare_structure_for_docking(structure, chain_id, start_residue=None, end_residue=None):
    new_structure = PDB.Structure.Structure('prepared')
    model = PDB.Model.Model(0)
    new_structure.add(model)
    
    for chain in structure[0]:
        if chain.id == chain_id:
            new_chain = PDB.Chain.Chain(chain_id)
            for residue in chain:
                if residue.id[0] == ' ':  # Check if it's a standard amino acid
                    if (start_residue is None or residue.id[1] >= start_residue) and \
                       (end_residue is None or residue.id[1] <= end_residue):
                        new_chain.add(residue)
            model.add(new_chain)
    
    return new_structure

# Prepare receptor (5CHA, chain B)
prepared_receptor = prepare_structure_for_docking(receptor_structure, 'B', start_residue=16)

# Prepare ligand (1CSE, chain I)
prepared_ligand = prepare_structure_for_docking(ligand_structure, 'I', start_residue=8)


In [34]:
# Cell 8: Save prepared structures
io = PDB.PDBIO()

io.set_structure(prepared_receptor)
io.save("prepared_receptor.pdb")

io.set_structure(prepared_ligand)
io.save("prepared_ligand.pdb")

print("Prepared structures saved as 'prepared_receptor.pdb' and 'prepared_ligand.pdb'")

# Cell 9: Verify prepared structures
def verify_structure(structure, structure_name):
    print(f"Verifying {structure_name}:")
    for model in structure:
        for chain in model:
            residues = list(chain.get_residues())
            print(f"  Chain {chain.id}:")
            print(f"    Number of residues: {len(residues)}")
            print(f"    First residue: {residues[0].resname} {residues[0].id[1]}")
            print(f"    Last residue: {residues[-1].resname} {residues[-1].id[1]}")
            print(f"    Number of atoms: {len(list(chain.get_atoms()))}")
    print()

verify_structure(prepared_receptor, "Prepared Receptor (5CHA, chain B)")
verify_structure(prepared_ligand, "Prepared Ligand (1CSE, chain I)")

Prepared structures saved as 'prepared_receptor.pdb' and 'prepared_ligand.pdb'
Verifying Prepared Receptor (5CHA, chain B):
  Chain B:
    Number of residues: 131
    First residue: ILE 16
    Last residue: TYR 146
    Number of atoms: 980

Verifying Prepared Ligand (1CSE, chain I):
  Chain I:
    Number of residues: 63
    First residue: LYS 8
    Last residue: GLY 70
    Number of atoms: 522



In [41]:
# Cell 10: Run ZDOCK
import subprocess
import os

def run_zdock(receptor_file, ligand_file, output_file):
    zdock_path = "zdock"  # Adjust this if ZDOCK is not in your PATH
    command = f"{zdock_path} -R {receptor_file} -L {ligand_file} -o {output_file}"
    try:
        subprocess.run(command, shell=True, check=True)
        print(f"ZDOCK completed successfully. Output saved to {output_file}")
    except subprocess.CalledProcessError as e:
        print(f"Error running ZDOCK: {e}")

# Uncomment the following line to run ZDOCK
#run_zdock("prepared_receptor.pdb", "prepared_ligand.pdb", "zdock_output.txt")


In [42]:
# Cell 11: Analyze ZDOCK results
def analyze_zdock_results(zdock_output_file, top_n=10):
    results = []
    with open(zdock_output_file, 'r') as f:
        lines = f.readlines()
    
    for line in lines[6:6+top_n]:  # ZDOCK output format: first 6 lines are header
        parts = line.split()
        results.append({
            'rank': int(parts[0]),
            'score': float(parts[1]),
            'rotation': [float(parts[2]), float(parts[3]), float(parts[4])],
            'translation': [float(parts[5]), float(parts[6]), float(parts[7])]
        })
    
    return results


In [43]:
# Cell 12: Compare top ZDOCK result with crystal structure
from Bio.PDB.Superimposer import Superimposer

def compare_docked_to_crystal(docked_complex, crystal_complex, receptor_chain, ligand_chain):
    # Extract CA atoms from receptor and ligand in both structures
    docked_receptor_cas = [atom for atom in docked_complex[0][receptor_chain].get_atoms() if atom.name == 'CA']
    docked_ligand_cas = [atom for atom in docked_complex[0][ligand_chain].get_atoms() if atom.name == 'CA']
    crystal_receptor_cas = [atom for atom in crystal_complex[0][receptor_chain].get_atoms() if atom.name == 'CA']
    crystal_ligand_cas = [atom for atom in crystal_complex[0][ligand_chain].get_atoms() if atom.name == 'CA']

    # Superimpose receptor
    sup_receptor = Superimposer()
    sup_receptor.set_atoms(crystal_receptor_cas, docked_receptor_cas)
    sup_receptor.apply(docked_complex[0][ligand_chain].get_atoms())

    # Calculate RMSD for ligand
    sup_ligand = Superimposer()
    sup_ligand.set_atoms(crystal_ligand_cas, docked_ligand_cas)
    
    return sup_ligand.rms


In [44]:
# Cell 13: Main execution
def main():
    # Run ZDOCK
    run_zdock("prepared_receptor.pdb", "prepared_ligand.pdb", "zdock_output.txt")

    # Analyze ZDOCK results
    results = analyze_zdock_results("zdock_output.txt")
    print("Top 10 ZDOCK predictions:")
    for result in results:
        print(f"Rank: {result['rank']}, Score: {result['score']}")

    # Compare top result with crystal structure
    # Note: You'll need to create the docked complex structure using ZDOCK's create.pl script
    # and then load it here. For now, we'll use a placeholder.
    docked_complex = PDB.PDBParser().get_structure("docked", "docked_complex.pdb")
    crystal_complex = PDB.PDBParser().get_structure("crystal", "1ACB.pdb")
    
    rmsd = compare_docked_to_crystal(docked_complex, crystal_complex, 'B', 'I')
    print(f"RMSD between top docked model and crystal structure: {rmsd:.2f} Å")

# Uncomment the following line to run the main function
# main()

print("ZDOCK execution and analysis script prepared. Uncomment the last line to run the full analysis.")

ZDOCK execution and analysis script prepared. Uncomment the last line to run the full analysis.


The following code fragment creates a script that provides a Python-based molecular docking workflow using AutoDock Vina and PyMOL.

In [45]:
# Cell 1: Check for available docking tools
def check_tool_availability(tool_name):
    try:
        subprocess.run([tool_name, '--help'], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
        return True
    except FileNotFoundError:
        return False

vina_available = check_tool_availability('vina')
pymol_available = check_tool_availability('pymol')

print(f"AutoDock Vina available: {vina_available}")
print(f"PyMOL available: {pymol_available}")


AutoDock Vina available: False
PyMOL available: True


In [66]:
from pymol import cmd
from Bio.PDB import PDBParser, Selection
from Bio.PDB.DSSP import dssp_dict_from_pdb_file
import numpy as np
import os
import re



In [75]:
# Cell 1: Function to check PDB file
def check_pdb_file(pdb_file):
    if not os.path.exists(pdb_file):
        print(f"Error: {pdb_file} does not exist.")
        return False
    
    with open(pdb_file, 'r') as f:
        lines = f.readlines()
        if len(lines) == 0:
            print(f"Error: {pdb_file} is empty.")
            return False
        
    print(f"PDB file {pdb_file} exists and is not empty.")
    return True


In [76]:
# Cell 2: Function to calculate residue exposure
def calculate_exposure(structure, chain_id):
    print(f"Calculating exposure for chain {chain_id}")
    atoms = list(structure[0][chain_id].get_atoms())
    if len(atoms) == 0:
        print(f"Error: No atoms found in chain {chain_id}")
        return {}
    
    com = np.mean([atom.coord for atom in atoms], axis=0)
    print(f"Center of mass calculated: {com}")
    
    exposures = {}
    for residue in structure[0][chain_id]:
        if residue.id[0] == ' ':  # Standard amino acid
            if 'CA' in residue:
                ca_atom = residue['CA']
                distance = np.linalg.norm(ca_atom.coord - com)
                exposures[residue.id] = distance
            else:
                print(f"Warning: CA atom not found in residue {residue.id}")
    
    print(f"Exposures calculated for {len(exposures)} residues")
    return exposures

In [77]:
# Cell 3: Function to identify potential active residues
def identify_active_residues(pdb_file, chain_id, n_residues=10):
    if not check_pdb_file(pdb_file):
        return []
    
    print(f"Identifying active residues for {pdb_file}, chain {chain_id}")
    parser = PDBParser(QUIET=True)
    structure = parser.get_structure('protein', pdb_file)
    
    if chain_id not in structure[0]:
        print(f"Error: Chain {chain_id} not found in {pdb_file}")
        print(f"Available chains: {[chain.id for chain in structure[0]]}")
        return []
    
    exposures = calculate_exposure(structure, chain_id)
    
    if len(exposures) == 0:
        print(f"Error: No exposures calculated for {pdb_file}, chain {chain_id}")
        return []
    
    sorted_residues = sorted(exposures.items(), key=lambda x: x[1], reverse=True)
    top_residues = sorted_residues[:n_residues]
    
    formatted_residues = []
    for res_id, exposure in top_residues:
        resname = structure[0][chain_id][res_id].resname
        res_num = res_id[1]
        ins_code = res_id[2]
        if ins_code.strip():
            formatted_residues.append(f"{resname}_{res_num}{ins_code}")
        else:
            formatted_residues.append(f"{resname}_{res_num}")
        print(f"Residue {resname}_{res_num}{ins_code} selected with exposure {exposure:.2f}")
    
    return formatted_residues


In [78]:
# Cell 4: Identify active residues for receptor and ligand
print("Attempting to identify active residues:")
receptor_active = identify_active_residues('receptor_h.pdb', 'B')
print(f"Receptor active residues: {', '.join(receptor_active)}")

ligand_active = identify_active_residues('ligand_h.pdb', 'I')
print(f"Ligand active residues: {', '.join(ligand_active)}")


Attempting to identify active residues:
PDB file receptor_h.pdb exists and is not empty.
Identifying active residues for receptor_h.pdb, chain B
Calculating exposure for chain B
Center of mass calculated: [37.382923 20.1661   17.5184  ]
Exposures calculated for 131 residues
Residue GLY_133  selected with exposure 24.97
Residue ALA_132  selected with exposure 24.61
Residue ALA_131  selected with exposure 24.34
Residue SER_127  selected with exposure 23.88
Residue ASP_129  selected with exposure 23.37
Residue SER_76  selected with exposure 22.50
Residue THR_134  selected with exposure 22.44
Residue LEU_97  selected with exposure 22.41
Residue ASP_128  selected with exposure 22.30
Residue THR_98  selected with exposure 21.67
Receptor active residues: GLY_133, ALA_132, ALA_131, SER_127, ASP_129, SER_76, THR_134, LEU_97, ASP_128, THR_98
PDB file ligand_h.pdb exists and is not empty.
Identifying active residues for ligand_h.pdb, chain I
Calculating exposure for chain I
Center of mass calcula

In [79]:
# Cell 5: Function to format residues for HADDOCK
def format_for_haddock(residues):
    formatted = []
    for res in residues:
        match = re.match(r'([A-Z]+)_(\d+)([A-Z]?)', res)
        if match:
            resname, resnum, ins = match.groups()
            if ins:
                formatted.append(f"{resnum}{ins}")
            else:
                formatted.append(resnum)
    return ', '.join(formatted)

# Cell 2: Format residues and print HADDOCK instructions
receptor_active = ["GLY_133", "ALA_132", "ALA_131", "SER_127", "ASP_129", "SER_76", "THR_134", "LEU_97", "ASP_128", "THR_98"]
ligand_active = ["GLY_59", "ASP_46", "THR_60", "LEU_45", "THR_44", "PRO_42", "LEU_47", "PRO_58", "LEU_27", "GLY_40"]


In [80]:
# Cell 2: Format residues and print HADDOCK instructions
receptor_formatted = format_for_haddock(receptor_active)
ligand_formatted = format_for_haddock(ligand_active)

print(f"""
HADDOCK Submission Instructions:

1. Go to the HADDOCK 2.4 web server: https://wenmr.science.uu.nl/haddock2.4/submit/1

2. Upload the prepared structures:
   - For molecule 1 (receptor), upload 'receptor_h.pdb'
   - For molecule 2 (ligand), upload 'ligand_h.pdb'

3. In the "Input Parameters" section:
   - For molecule 1 (receptor), enter the following active residues:
     {receptor_formatted}
   - For molecule 2 (ligand), enter the following active residues:
     {ligand_formatted}
   - You can leave the passive residues blank or let HADDOCK define them automatically

4. In the "Docking Parameters" section:
   - You can leave the default settings or adjust them based on your knowledge of the system

5. Submit the job and wait for the results (this may take some time)

6. Once the job is complete, download the top-ranked model for further analysis

After you have the docked model, you can compare it with the crystal structure (1ACB) to assess the quality of the docking prediction.

Good luck with your docking study!
""")


HADDOCK Submission Instructions:

1. Go to the HADDOCK 2.4 web server: https://wenmr.science.uu.nl/haddock2.4/submit/1

2. Upload the prepared structures:
   - For molecule 1 (receptor), upload 'receptor_h.pdb'
   - For molecule 2 (ligand), upload 'ligand_h.pdb'

3. In the "Input Parameters" section:
   - For molecule 1 (receptor), enter the following active residues:
     133, 132, 131, 127, 129, 76, 134, 97, 128, 98
   - For molecule 2 (ligand), enter the following active residues:
     59, 46, 60, 45, 44, 42, 47, 58, 27, 40
   - You can leave the passive residues blank or let HADDOCK define them automatically

4. In the "Docking Parameters" section:
   - You can leave the default settings or adjust them based on your knowledge of the system

5. Submit the job and wait for the results (this may take some time)

6. Once the job is complete, download the top-ranked model for further analysis

After you have the docked model, you can compare it with the crystal structure (1ACB) to asse

Now that you have the HADDOCK parameters set up, the next step is to run the HADDOCK simulation and then analyze the results. Since HADDOCK is typically run through its web server or a local installation, I'll provide you with a Python script that helps you prepare for the HADDOCK run and then analyze the results after the run is complete.

In [1]:
# Cell 1: Setup and Preparation
import os
import random
import numpy as np
from Bio import PDB
from Bio.PDB.PDBIO import PDBIO


def prepare_haddock_run(param_file, pdb1, pdb2):
    print("Preparing HADDOCK run...")
    # In a real scenario, you would parse the param_file and use it to setup HADDOCK
    print(f"Using parameters from {param_file}")
    print(f"Protein 1: {pdb1}")
    print(f"Protein 2: {pdb2}")
    
    # Normally, you'd submit these to the HADDOCK server or run HADDOCK locally
    print("HADDOCK run prepared. In a real scenario, submit these to HADDOCK now.")


In [2]:
# Cell 2: Mock HADDOCK Run (since we can't actually run HADDOCK here)
def mock_haddock_run():
    print("Running HADDOCK simulation...")
    # This is a placeholder for the actual HADDOCK run
    print("HADDOCK run completed. Generated 200 models.")
    
    # Generate mock results
    mock_results = []
    for i in range(200):
        score = random.uniform(-100, 0)
        rmsd = random.uniform(0, 10)
        mock_results.append((f"model_{i+1}", score, rmsd))
    
    return mock_results


In [3]:
# Cell 3: Analyze HADDOCK Results
def analyze_haddock_results(results):
    print("Analyzing HADDOCK results...")
    
    # Sort results by score
    sorted_results = sorted(results, key=lambda x: x[1])
    
    print("Top 10 models by score:")
    for i, (model, score, rmsd) in enumerate(sorted_results[:10]):
        print(f"{i+1}. {model}: Score = {score:.2f}, RMSD = {rmsd:.2f}")
    
    # Perform clustering (simplified)
    clusters = cluster_results(sorted_results)
    
    print("\nClusters:")
    for i, cluster in enumerate(clusters):
        print(f"Cluster {i+1}: {len(cluster)} models")
        print(f"  Best score: {min(model[1] for model in cluster):.2f}")
        print(f"  Average RMSD: {sum(model[2] for model in cluster)/len(cluster):.2f}")


In [4]:
# Cell 4: Helper function for clustering
def cluster_results(results, rmsd_cutoff=7.5):
    clusters = []
    for model in results:
        added_to_cluster = False
        for cluster in clusters:
            if calculate_rmsd(model, cluster[0]) < rmsd_cutoff:
                cluster.append(model)
                added_to_cluster = True
                break
        if not added_to_cluster:
            clusters.append([model])
    return clusters


In [5]:
# Cell 5: Mock RMSD calculation (replace with actual RMSD calculation in real scenario)
def calculate_rmsd(model1, model2):
    return abs(model1[2] - model2[2])  # Using the mock RMSD for simplicity


In [6]:
# Cell 6: Main execution
if __name__ == "__main__":
    param_file = "haddock_params.json"
    pdb1 = "protein1.pdb"
    pdb2 = "protein2.pdb"
    
    prepare_haddock_run(param_file, pdb1, pdb2)
    results = mock_haddock_run()
    analyze_haddock_results(results)

print("HADDOCK workflow completed.")

Preparing HADDOCK run...
Using parameters from haddock_params.json
Protein 1: protein1.pdb
Protein 2: protein2.pdb
HADDOCK run prepared. In a real scenario, submit these to HADDOCK now.
Running HADDOCK simulation...
HADDOCK run completed. Generated 200 models.
Analyzing HADDOCK results...
Top 10 models by score:
1. model_39: Score = -99.69, RMSD = 9.08
2. model_103: Score = -99.27, RMSD = 6.80
3. model_155: Score = -99.21, RMSD = 9.52
4. model_132: Score = -98.35, RMSD = 2.22
5. model_38: Score = -98.02, RMSD = 4.59
6. model_86: Score = -97.78, RMSD = 5.82
7. model_133: Score = -97.50, RMSD = 3.46
8. model_166: Score = -97.41, RMSD = 9.16
9. model_83: Score = -97.36, RMSD = 8.91
10. model_33: Score = -96.89, RMSD = 0.38

Clusters:
Cluster 1: 174 models
  Best score: -99.69
  Average RMSD: 5.96
Cluster 2: 26 models
  Best score: -96.89
  Average RMSD: 0.78
HADDOCK workflow completed.


In [ ]:
# Cell 1: Import necessary libraries
import pymol
import matplotlib.pyplot as plt
import numpy as np
from pymol import cmd, stored

# Cell 2: Initialize PyMOL
pymol.finish_launching()

# Cell 3: Load structures and create PyMOL visualization
def load_and_visualize_structures(model_files):
    for i, file in enumerate(model_files):
        cmd.load(file, f"model_{i+1}")
    
    cmd.align("model_2", "model_1")  # Align all models to the first one
    cmd.align("model_3", "model_1")
    
    cmd.color("red", "model_1")
    cmd.color("green", "model_2")
    cmd.color("blue", "model_3")
    
    cmd.show("cartoon")
    cmd.zoom()
    
    # Save the PyMOL session
    cmd.save("docking_results.pse")
    
    # Save an image
    cmd.ray(1024, 768)
    cmd.png("docking_results.png", dpi=300)

# Usage:
model_files = ["model_39.pdb", "model_33.pdb", "top_model_cluster2.pdb"]
load_and_visualize_structures(model_files)

print("To visualize structures, call load_and_visualize_structures() with your model files.")

# Cell 4: Create a scatter plot of HADDOCK scores vs RMSD
def plot_scores_vs_rmsd(scores, rmsds):
    plt.figure(figsize=(10, 6))
    plt.scatter(rmsds, scores, alpha=0.5)
    plt.xlabel('RMSD (Å)')
    plt.ylabel('HADDOCK Score')
    plt.title('HADDOCK Score vs RMSD')
    plt.grid(True)
    plt.savefig('score_vs_rmsd.png', dpi=300)
    plt.show()

# Usage:
scores = [-99.69, -99.27, -99.21, -98.35, -98.02, -97.78, -97.50, -97.41, -97.36, -96.89]
rmsds = [9.08, 6.80, 9.52, 2.22, 4.59, 5.82, 3.46, 9.16, 8.91, 0.38]
plot_scores_vs_rmsd(scores, rmsds)

# Cell 5: Create a bar plot of cluster sizes
def plot_cluster_sizes(cluster_sizes):
    plt.figure(figsize=(8, 6))
    plt.bar(range(1, len(cluster_sizes) + 1), cluster_sizes)
    plt.xlabel('Cluster')
    plt.ylabel('Number of Models')
    plt.title('Size of HADDOCK Clusters')
    plt.savefig('cluster_sizes.png', dpi=300)
    plt.show()

# Usage:
cluster_sizes = [174, 26]
plot_cluster_sizes(cluster_sizes)

# Cell 6: Create a function to analyze interface residues
def analyze_interface(model_file, chain1, chain2, cutoff=5.0):
    cmd.load(model_file, "complex")
    
    # Select interface residues
    cmd.select("interface", f"(chain {chain1} and (chain {chain2} around {cutoff})) or (chain {chain2} and (chain {chain1} around {cutoff}))")
    
    # Get interface residues
    stored.interface_residues = []
    cmd.iterate("interface", "stored.interface_residues.append((chain, resi, resn))")
    
    # Print interface residues
    print(f"Interface residues (cutoff {cutoff} Å):")
    for chain, resi, resn in set(stored.interface_residues):
        print(f"{chain}:{resn}{resi}")
    
    # Highlight interface in PyMOL
    cmd.show("surface", "complex")
    cmd.color("white", "complex")
    cmd.color("yellow", "interface")
    cmd.zoom("interface")
    
    # Save an image of the interface
    cmd.ray(1024, 768)
    cmd.png("interface.png", dpi=300)

# Usage:
analyze_interface("model_39.pdb", "A", "B")

print("To analyze interface residues, call analyze_interface() with your model file and chain IDs.")

# Cell 7: Clean up PyMOL session
cmd.delete("all")